In [7]:
import magellan as mg
import pandas as pd

In [8]:
dblp = mg.read_csv('DBLP_cleaned.csv', key='id')

In [9]:
acm = mg.read_csv('ACM_cleaned.csv', key='id')

In [29]:
for c in acm.columns:
    print sum(dblp[c].apply(pd.isnull))

0
0
0
0
0


In [32]:
gold = mg.read_csv('dblp-acm_gold.csv', key='pairId')

In [36]:
gold.columns = ['id', 'ltable.id', 'rtable.id', 'label']

In [38]:
mg.view(gold)

In [50]:
g = mg.create_mtable(gold, key='id', ltable=dblp, rtable=acm, foreign_key_ltable='ltable.id', foreign_key_rtable='rtable.id')

In [52]:
g.get_key()

'id'

In [53]:
ab = mg.AttrEquivalenceBlocker()

In [62]:
C = ab.block_candset(g, 'venue', 'venue')

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 0.266 sec


In [63]:
len(C)

0